In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_36.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


You arrived at an old self-storage facility by invitation from the manager, who has an storage room up for auction due to the death of its owner. It is not your first rodeo with these forms of auctions, like a box of chocolate, no one knows of the potential treasure that hides behind the cold roller shutter. As you and other interested buyers gathered around the auctioneer, a worker carrying a bolt cutter snapped off the rusty lock, unveiling a storage room cluttered with various old furniture and antiquities. Without any opportunity for the bidders to take a clear look, the auctioneer began the bid at $50. 

What is your bid?

possible actions:
1. $100.
2. $150.
3. $200.


I select option 3. TURN: 1 


You confidently raise your hand, projecting your bid of $200 into the crowd. The auctioneer nods in your direction, acknowledging your bid. The atmosphere becomes tense as other bidders weigh their options, their eyes darting between the hidden treasures of the storage unit and you. 

The auctioneer's voice cuts through the murmurs, "We have $200! Do I hear $250?" 

possible actions:
1. Raise the bid to $250.
2. Hold your current bid and observe the other bidders.
3. Drop out of the bidding.

I select option 2. TURN: 2 


You decide to hold your current bid at $200, observing the reactions of the other bidders closely. A moment of silence hangs in the air as everyone seems to be calculating their next move. Then, from the back of the crowd, a voice calls out, "I'll go $250!"

The auctioneer points towards the new bidder, "We have $250! Can I get $300?" He scans the crowd, looking for any signs of further interest.

possible actions:
1. Raise the bid to $300.
2. Continue to hold your bid and watch the situation unfold.
3. Drop out of the bidding.

I select option 1. TURN: 3 


You step back into the fray, raising your bid to $300 with a determined nod. The auctioneer's eyes light up as he acknowledges your bid, "We have $300 from the gentleman here! Do I hear $350?"

The crowd murmurs, and you can feel the tension as the number of active bidders starts to dwindle. It's clear that your aggressive bidding has made some of the other potential buyers hesitant to continue.

possible actions:
1. Prepare to raise your bid further if challenged.
2. Maintain your current bid and closely observe the reactions of the remaining bidders.
3. Consider carefully if you're willing to go any higher, given your financial limit.

I select option 2. TURN: 4 


You maintain your current bid at $300, watching the remaining bidders with a keen eye. The auctioneer's call for $350 echoes unanswered, and the tension among the bidders is palpable. After a brief pause, the auctioneer repeats, "We're at $300, going once... going twice..."

In these final moments, you scrutinize the faces of the other bidders, trying to gauge if anyone will make a last-second challenge to your bid.

possible actions:
1. Mentally prepare to celebrate your potential win.
2. Stay focused and ready to counter any last-minute bids.
3. Start planning what you'll do first if you win the storage room.

I select option 2. TURN: 5 


You stay focused, ready to counter any last-minute bids that might come your way. The auctioneer's voice fills the air, "Going twice... and sold to the gentleman for $300!"

The crowd begins to disperse, some with looks of disappointment, others with relief. You've won the auction, and the anticipation of discovering what lies within the storage room fills you with a mix of excitement and nervousness. The auctioneer approaches you to finalize the paperwork and collect the payment.

round summary: Kyle Jackson successfully won the auction for the storage room with a bid of $300, carefully navigating through the bidding process and outlasting the competition.